In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'
#makes line plots of spatially-averaged seasonal amount of top 5% precip 

In [ ]:
if 1==1:#part 2: read the obs at eligible stations into a better format
    yr_start=1979
    yr_end=2019
    tuples=[]
    years=[]
    months=[]
    days1=[]
    lats1=[]
    lons1=[]
    elevs1=[]
    observations=[]
    dates=[]
    station_names_list=[]
    stations_list=[]
    dp=xr.open_dataset(dir+'station_numbers_95.nc')
    stations=dp.stations.values.tolist()
    lats=dp.lats.values.tolist()
    lons=dp.lons.values.tolist()
    elevs=dp.elevs.values.tolist()
    station_names=dp.station_names.values.tolist()
    points=zip(lons,lats)
    for i in range(len(stations)):
        print(str(i+1)+'/'+str(len(stations)))
        lat=lats[i]
        lon=lons[i]
        elev=elevs[i]
        station_name=station_names[i]
        station=stations[i]
        filename=stations[i]
        ds1=pd.read_csv(filename,header=None)
        nancount=0
        qcount=0
        yescount=0
        for row1 in ds1.iterrows():
            row_neg_space=row1[1][0].replace('-',' -')
            row_basic=row1[1][0]
            a=row_neg_space.split()
            code=a[0][-4:]
            if code=='PRCP':
                year=int(a[0][11:15])
                if yr_start<=year<=yr_end:
                    month=int(a[0][15:17])
                    if month in [1,3,5,7,8,10,12]:
                        days=31
                    if month in [4,6,9,11]:
                        days=30
                    if month==2 and year%4==0:
                        days=29
                    if month==2 and year%4!=0:
                        days=28

                    for i in range(days):
                        #print(row_basic)
                        mflag=row_basic[26+8*i]
                        qflag=row_basic[27+8*i]
                        sflag=row_basic[28+8*i]
                        obs=float(row_basic[21+8*i:26+8*i])
                    
                        if obs==-9999.0:
                            pass
                        elif qflag!=' ':
                            pass
                        else:
                            years.append(year)
                            months.append(month)
                            days1.append(i)
                            observations.append(obs/10.)#convert from tenths of mm to mm
                            lats1.append(lat)
                            lons1.append(lon)
                            elevs1.append(elev)
                            date=dt.datetime(year,month,i+1,6)
                            dates.append(date)
                            station_names_list.append(station_name)
                            stations_list.append(station)
obs=observations
lats=lats1
lons=lons1
stations=stations_list

In [ ]:
ds1.close()
ds1=xr.open_dataset(dir+'station_numbers_95.nc')
lats_station=ds1['lats']#use these lists for plotting
lons_station=ds1['lons']
stations_station=ds1['stations']

thresholds_winter_99=ds1['thresholds_winter_99']/10.
thresholds_spring_99=ds1['thresholds_spring_99']/10.
thresholds_summer_99=ds1['thresholds_summer_99']/10.
thresholds_fall_99=ds1['thresholds_fall_99']/10.

thresholds_winter_95=ds1['thresholds_winter_95']/10.
thresholds_spring_95=ds1['thresholds_spring_95']/10.
thresholds_summer_95=ds1['thresholds_summer_95']/10.
thresholds_fall_95=ds1['thresholds_fall_95']/10.

thresholds_winter_90=ds1['thresholds_winter_90']/10.
thresholds_spring_90=ds1['thresholds_spring_90']/10.
thresholds_summer_90=ds1['thresholds_summer_90']/10.
thresholds_fall_90=ds1['thresholds_fall_90']/10.

In [ ]:
yrs_neusa=np.arange(1979,2020,1)
zipped=list(zip(dates,stations_list,lats1,lons1,observations))

print(len(zipped))
sum_years_array_99=[]
sum_years_array_95=[]
sum_years_array_90=[]
for i in range(len(stations1)):
    print(i)
    threshold=thresholds[i]
    station=stations1[i].values
    select_station_99=[x for x in zipped if x[1]==station and pd.to_datetime(x[0]).month in [1,2,12] and x[4]>=thresholds_winter_99[i]]
    select_station_95=[x for x in zipped if x[1]==station and pd.to_datetime(x[0]).month in [1,2,12] and x[4]>=thresholds_winter_95[i]]
    select_station_90=[x for x in zipped if x[1]==station and pd.to_datetime(x[0]).month in [1,2,12] and x[4]>=thresholds_winter_90[i]]

    sum_years_list_99=[]
    sum_years_list_95=[]
    sum_years_list_90=[]
    
    for j in range(1,len(yrs_neusa)):
        year=yrs_neusa[j]
        
        if 1==0:
            select_extremes_year_99=[x[4] for x in select_station_99 if pd.to_datetime(x[0]).year==year]
            select_extremes_year_95=[x[4] for x in select_station_95 if pd.to_datetime(x[0]).year==year]
            select_extremes_year_90=[x[4] for x in select_station_90 if pd.to_datetime(x[0]).year==year]
        if 1==1:
            year_prev=yrs_neusa[j-1]
            select_extremes_year_99_prev=[x[4] for x in select_station_99 if pd.to_datetime(x[0]).year==year_prev and pd.to_datetime(x[0]).month==12]
            select_extremes_year_99_current=[x[4] for x in select_station_99 if pd.to_datetime(x[0]).year==year and pd.to_datetime(x[0]).month in [1,2]]
            select_extremes_year_99=select_extremes_year_99_prev+select_extremes_year_99_current
            
            select_extremes_year_95_prev=[x[4] for x in select_station_95 if pd.to_datetime(x[0]).year==year_prev and pd.to_datetime(x[0]).month==12]
            select_extremes_year_95_current=[x[4] for x in select_station_95 if pd.to_datetime(x[0]).year==year and pd.to_datetime(x[0]).month in [1,2]]
            select_extremes_year_95=select_extremes_year_95_prev+select_extremes_year_95_current
            
            select_extremes_year_90_prev=[x[4] for x in select_station_90 if pd.to_datetime(x[0]).year==year_prev and pd.to_datetime(x[0]).month==12]
            select_extremes_year_90_current=[x[4] for x in select_station_90 if pd.to_datetime(x[0]).year==year and pd.to_datetime(x[0]).month in [1,2]]
            select_extremes_year_90=select_extremes_year_90_prev+select_extremes_year_90_current
        
        sum_years_list_99.append(sum(select_extremes_year_99))
        sum_years_list_95.append(sum(select_extremes_year_95))
        sum_years_list_90.append(sum(select_extremes_year_90))
        
    sum_years_array_99.append(sum_years_list_99)
    sum_years_array_95.append(sum_years_list_95)
    sum_years_array_90.append(sum_years_list_90)
    
time_series_99=[]
time_series_95=[]
time_series_90=[]
for i in range(len(yrs_neusa)):
    all_extreme_sums_year_99=[x[i] for x in sum_years_array_99]
    all_extreme_sums_year_95=[x[i] for x in sum_years_array_95]
    all_extreme_sums_year_90=[x[i] for x in sum_years_array_90]
    
    year_sum_99=sum(all_extreme_sums_year_99)
    year_sum_95=sum(all_extreme_sums_year_95)
    year_sum_90=sum(all_extreme_sums_year_90)
    
    time_series_99.append(year_sum_99)
    time_series_95.append(year_sum_95)
    time_series_90.append(year_sum_90)

In [ ]:
time_series_99=[]
time_series_95=[]
time_series_90=[]
for i in range(len(yrs_neusa)-1):
    all_extreme_sums_year_99=[x[i] for x in sum_years_array_99]
    all_extreme_sums_year_95=[x[i] for x in sum_years_array_95]
    all_extreme_sums_year_90=[x[i] for x in sum_years_array_90]
    
    year_sum_99=sum(all_extreme_sums_year_99)
    year_sum_95=sum(all_extreme_sums_year_95)
    year_sum_90=sum(all_extreme_sums_year_90)
    
    time_series_99.append(year_sum_99)
    time_series_95.append(year_sum_95)
    time_series_90.append(year_sum_90)

In [ ]:
print(time_series_99)

In [ ]:
#@author: Michael Schramm on GitHub
#This function is derived from code originally posted by Sat Kumar Tomer
#(satkumartomer@gmail.com)
#See also: http://vsp.pnnl.gov/help/Vsample/Design_Trend_Mann_Kendall.htm
from scipy.stats import norm
import scipy.stats as st
def mk_test(x, alpha=0.05):
    n = len(x)

    # calculate S
    s = 0
    for k in range(n-1):
        for j in range(k+1, n):
            s += np.sign(x[j] - x[k])

    # calculate the unique data
    unique_x, tp = np.unique(x, return_counts=True)
    g = len(unique_x)

    # calculate the var(s)
    if n == g:  # there is no tie
        var_s = (n*(n-1)*(2*n+5))/18
    else:  # there are some ties in data
        var_s = (n*(n-1)*(2*n+5) - np.sum(tp*(tp-1)*(2*tp+5)))/18

    if s > 0:
        z = (s - 1)/np.sqrt(var_s)
    elif s < 0:
        z = (s + 1)/np.sqrt(var_s)
    else: # s == 0:
        z = 0

    # calculate the p_value
    p = 2*(1-norm.cdf(abs(z)))  # two tail test
    h = abs(z) > norm.ppf(1-alpha/2)

    if (z < 0) and h:
        trend = 'decreasing'
    elif (z > 0) and h:
        trend = 'increasing'
    else:
        trend = 'no trend'

    return trend, h, p, z

In [ ]:
#years_sums_list
#years_freqs_list
#years_ints_list

if 1==0:
    time_series_99=[x/1000. for x in time_series_99]
    time_series_95=[x/1000. for x in time_series_95]
    time_series_90=[x/1000. for x in time_series_90]
    
mk_result_99=mk_test(time_series_99,alpha=0.05)[0]
mk_result_95=mk_test(time_series_95,alpha=0.05)[0]
mk_result_90=mk_test(time_series_90,alpha=0.05)[0]

linreg_99=st.linregress(yrs_neusa[1:],time_series_99)
slope_95=linreg_99[0]
p_value_95=linreg_99[3]

linreg_95=st.linregress(yrs_neusa[1:],time_series_95)
slope_95=linreg_95[0]
p_value_95=linreg_95[3]

linreg_90=st.linregress(yrs_neusa[1:],time_series_90)
slope_90=linreg_90[0]
p_value_90=linreg_90[3]

In [ ]:
if mk_result_99 in ['increasing','decreasing']:
    mk99_str='MK-test = significant'
else:
    mk99_str='MK-test = insignificant'
    
if mk_result_95 in ['increasing','decreasing']:
    mk95_str='MK-test = significant'
else:
    mk95_str='MK-test = insignificant'
    
if mk_result_90 in ['increasing','decreasing']:
    mk90_str='MK-test = significant'
else:
    mk90_str='MK-test = insignificant'
    

if linreg_99[1]<0:
    eq_99='y='+str(round(linreg_99[0],2))+'x-'+str(round(abs(linreg_99[1]),2))+', p='+str(round(linreg_99[3],4))
else:
    eq_99='y='+str(round(linreg_99[0],2))+'x+'+str(round(linreg_99[1],2))+', p='+str(round(linreg_99[3],4))
    
if linreg_95[1]<0:
    eq_95='y='+str(round(linreg_95[0],2))+'x-'+str(round(abs(linreg_95[1]),2))+', p='+str(round(linreg_95[3],4))
else:
    eq_95='y='+str(round(linreg_95[0],2))+'x+'+str(round(linreg_95[1],2))+', p='+str(round(linreg_95[3],4))
    
if linreg_90[1]<0:
    eq_90='y='+str(round(linreg_90[0],2))+'x-'+str(round(abs(linreg_90[1]),2))+', p='+str(round(linreg_90[3],4))
else:
    eq_90='y='+str(round(linreg_90[0],2))+'x+'+str(round(linreg_90[1],2))+', p='+str(round(linreg_90[3],4))
    

label_99='99$^{th}$ ('+eq_99+') | '+mk99_str
label_95='95$^{th}$ ('+eq_95+') | '+mk95_str
label_90='90$^{th}$ ('+eq_90+') | '+mk90_str

fig=plt.figure(figsize=(12,12))
ax=plt.subplot(1,1,1)
ax.tick_params(labelsize=24)
ax.set_title('Winter',fontsize=42,pad=10)
ax.set_ylabel('10$^{3}$ mm yr$^{-1}$',labelpad=10,color='k',fontsize=24)
ax.plot(yrs_neusa[1:],time_series_99,linewidth=1.5,linestyle='--',color='m',label=label_99)
ax.plot(yrs_neusa[1:],time_series_95,linewidth=1.5,linestyle='--',color='r',label=label_95)
ax.plot(yrs_neusa[1:],time_series_90,linewidth=1.5,linestyle='--',color='orange',label=label_90)

#ax.plot(yrs_neusa,[x*linreg_99[0]+linreg_99[1] for x in yrs_neusa],linewidth=5.0,color='m')
ax.plot(yrs_neusa[1:],[x*linreg_95[0]+linreg_95[1] for x in yrs_neusa[1:]],linewidth=3.0,color='darkred')
ax.plot(yrs_neusa[1:],[x*linreg_90[0]+linreg_90[1] for x in yrs_neusa[1:]],linewidth=3.0,color='darkorange')
ax.set_ylim(0,90)
ax.set_xlabel('Year',fontsize=24,labelpad=15)
plt.legend(loc='upper left',fontsize=17)
plt.show()

In [ ]:
fig.savefig(dir+'climo_seasonal_lines_winter.png')